In [33]:
# GOOGLE COLLAB USERS ONLY
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# IGNORE THE CONTENT OF THIS CELL
# tf.compat.v1.disable_eager_execution()

In [34]:
tf.__version__

'2.0.0'

## Step 1: The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [35]:
path_to_file = 'shakespeare.txt'

In [36]:
text = open(path_to_file, 'r').read()

In [37]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters

In [38]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [39]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [40]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [41]:
ind_to_char = np.array(vocab)

In [42]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [43]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [44]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

We now have a mapping we can use to go back and forth from characters to numerics.

In [45]:
sample = text[:20]
sample

'\n                   '

In [46]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [47]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [48]:
line = "From fairest creatures we desire increase"

In [49]:
len(line)

41

In [50]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [51]:
len(part_stanza)

131

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [52]:
seq_len = 120

In [53]:
total_num_seq = len(text)//(seq_len+1)

In [54]:
total_num_seq

45005

In [55]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

# for i in char_dataset.take(500):
#      print(ind_to_char[i.numpy()])

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [56]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [57]:
sequences

<BatchDataset shapes: (121,), types: tf.int64>

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [58]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [59]:
dataset = sequences.map(create_seq_targets)

In [60]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [61]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [62]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model

We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [63]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [65]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [66]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [67]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [68]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [69]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [70]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.



In [71]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [72]:
example_batch_predictions

<tf.Tensor: id=5809, shape=(128, 120, 84), dtype=float32, numpy=
array([[[ 0.00560492,  0.00246352, -0.00659057, ..., -0.00373563,
         -0.00622196,  0.00320968],
        [-0.00224962,  0.00241033, -0.00256048, ...,  0.00186655,
         -0.0087444 ,  0.00349575],
        [-0.00132393,  0.00756752, -0.00092927, ...,  0.00668064,
         -0.0078537 , -0.000584  ],
        ...,
        [ 0.00156174,  0.0057579 , -0.00258742, ...,  0.00197547,
         -0.00561197,  0.01075896],
        [-0.00226213,  0.002961  ,  0.00334379, ...,  0.00182662,
         -0.00176757,  0.00859085],
        [-0.00360303,  0.00855371,  0.00500408, ..., -0.00420807,
          0.00428178, -0.00217624]],

       [[-0.00127057, -0.00620704, -0.00436888, ..., -0.00148506,
         -0.00421145,  0.00163425],
        [-0.00196829, -0.0058482 ,  0.00133853, ...,  0.00415808,
          0.00369589, -0.00044955],
        [-0.00217744, -0.0040452 ,  0.00509485, ...,  0.00069148,
         -0.00278963,  0.00255138],
  

In [73]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [74]:
sampled_indices

<tf.Tensor: id=5815, shape=(120, 1), dtype=int64, numpy=
array([[64],
       [29],
       [ 5],
       [ 5],
       [ 0],
       [60],
       [57],
       [65],
       [56],
       [76],
       [ 2],
       [15],
       [22],
       [ 3],
       [14],
       [17],
       [32],
       [ 6],
       [46],
       [28],
       [25],
       [31],
       [25],
       [ 2],
       [55],
       [ 5],
       [67],
       [64],
       [27],
       [77],
       [29],
       [41],
       [19],
       [58],
       [82],
       [51],
       [81],
       [27],
       [32],
       [43],
       [81],
       [63],
       [61],
       [80],
       [33],
       [73],
       [ 2],
       [22],
       [62],
       [12],
       [11],
       [ 8],
       [50],
       [74],
       [55],
       [54],
       [34],
       [51],
       [74],
       [41],
       [82],
       [66],
       [14],
       [52],
       [47],
       [76],
       [10],
       [46],
       [59],
       [40],
       [13],
       [60],
       

In [75]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [76]:
sampled_indices

array([64, 29,  5,  5,  0, 60, 57, 65, 56, 76,  2, 15, 22,  3, 14, 17, 32,
        6, 46, 28, 25, 31, 25,  2, 55,  5, 67, 64, 27, 77, 29, 41, 19, 58,
       82, 51, 81, 27, 32, 43, 81, 63, 61, 80, 33, 73,  2, 22, 62, 12, 11,
        8, 50, 74, 55, 54, 34, 51, 74, 41, 82, 66, 14, 52, 47, 76, 10, 46,
       59, 40, 13, 60,  1, 17, 68, 36, 48, 16, 80, 64,  6, 82, 12, 23, 14,
       16, 29, 21, 35, 39, 48, 17, 28, 22, 83, 20, 14, 18, 41,  2, 36, 42,
        2, 10,  4, 18, 11, 75, 22, 58, 45, 10, 40, 23, 13, 63, 16, 41, 77,
       23])

In [77]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))


Given the input seq: 

ster's married; there's news for
    you; you have a new mistress.
  PAROLLES. I most unfeignedly beseech your lordship 


Next Char Predictions: 

iD''
ebjau!4;"36G(UC?F?!`'liBvDP8c|ZzBGRzhfyHr!;g10,Ys`_IZsP|k3[Vu.UdO2e 6mKW5yi(|1<35D:JNW6C;}937P!KQ!.&70t;cT.O<2h5Pv<


After confirming the dimensions are working, let's train our network!

In [78]:
epochs = 30

In [79]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
    240/Unknown - 1142s 5s/step - loss: 2.7523

KeyboardInterrupt: 

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [80]:
model.save('shakespeare_gen.h5') 

In [81]:
from tensorflow.keras.models import load_model

In [90]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [91]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_4 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [84]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [85]:
print(generate_text(model,"flower",gen_size=1000))

flowerk!
  HICOSS. Hatlt pivesnith
    The  fore mey, and os theond, wave goo ne lastise;
    FO ves ik wathsr fakioresson lome.
  R AG SORD Hape sorw ae muves s wayest y ar beponse; ga; mas ine;
    Thove rndise IAvane.
             andin t of halll'd, y your, thes ande utrouf,
    And he hat the Noons dave waer thoun libk;
    Adtsoll so podtour ior bes it foreaferam.   foo mey youcer daad, ut thew!- fou! meill! Walif od here
    Heaead, I hand be Fanctiscine.
    Sid ned ou corting, of afk, os vitner shiy sou thay mo therrnde ig nof ho o'd mee menteblofen
    Thess hom  o peyre! see mestewhame-I sot buck ghos
    no m'sprrem?
  A in fasash nt wHat sik the foor; wo ward  purirt incheicr the s pavevr, and bert-In gror madt dll. Thot in thim- Bet, padin
    la souve'd tho reprit atllser filk, bik so prithe. Sur yout dlave
    Oxkerf Hitthitg, 'n they ined miswes wit iccorb;
    UR ENw Peis in bed,
    He, sull prerinmans yow dey, lo kiren;] s yureme be Dilus Mas thrurer at he wathe  to